# 10:50 AM: Automate AWS With Python
[Moshe Zadka](https://orbifold.xyz/)

Why automate?
* Reproducibility + functionality

Root account
* your email/password

IAM account
* belongs to an AWS account
* does not have to have a password
* 2FA is probabily a good idea
* meant for friendly users/human users

API Key
* optional
* 0-2 accounts
* for robot users

> ** There is no reason to log in as root unless you'ev lost the credentials **


**Consolidated billing**
* Create master account
* create sub-accounts
* Easy to separate out


Corporate awesomeness
* Only admin/finance have access to master account: only for billing purposes
* No resources in the master account
* Production account for user-facing
* Staging account
* Build/test account
* Pets

Corporate policies
* Prod/stage/build - all resources automated, tagged with reasons
* Pets: tagged with dev name, garbage collect by default

Generating API Key
* Admin permission is needed
* Saing secrets is hard
* Keys have a publically sharable part (good for debugging)

Rotating API keys
* Up to 2 keys per user
* Generate 2nd, deploy, invalidate old
* Invalidation is reversible
* If no problems, delete

Regions and availability zones
* Regions: geographical areas
* Availability zones: data centers

EC2 Instances
* Virtual machines
* Basic compute usnit
* Size, cost varies

Use cloudinit python pckage that grabs instance metadata
* Configures machine, like ssh keys

Keypair management
* Generate pair on AWS
* Upload public
* Per-region name

> Do not ssh in until you confirm with boto3

AMI builds
* "Amazon machine unit"
* Ready made ones
* Make your own
* Can share across accounts
* Avoid secrets across acounts

Why build your own?
* Security updates
* Sepcific software
* Automate with salt

Cloud formation
* Templates are JSON files
* Can be parameterized
* Can be self-referential
* `json.dumps` works
* Try using troposphere

Can use the json to build a fresh stack.
`client.create_stack("MyStack", TemplateBody=cft)`

Updating a stack: creating a new JSON
* New JSON
* Create a change-set
* Optional: eyeball check
* Apply a change-set
* VPC in Cloud Formation
* Separating stacks with VPCs; easy to duplicate, can build whole networks (VPC, Subents, Security groups)

Summary
> no root logins

> cloud formation for everything (or terraform)

> automate everyhing

Snapshot management
* Debugging
* Can use for a halfway automated backup solution

# 11:30 AM: When the abyss gazes back: staring down Python's surprising internals

[David Wolever](https://twitter.com/wolever)


** Motivation: odd behavior **
[Why is tuple membership faster than equality?](http://stackoverflow.com/questions/28885132/why-is-x-in-x-faster-than-x-x)

ipython's timeit magic is wonderful.

`%timeit`

```python
dis.disassmble
```
Allows you to see the bytecode.

In [16]:
import dis
what = "World"
def say_hello1():
    msg = "Hello, %s" %(what,)
    print(msg)

In [17]:
dis.disassemble(say_hello1.__code__)

  4           0 LOAD_CONST               1 ('Hello, %s')
              3 LOAD_GLOBAL              0 (what)
              6 BUILD_TUPLE              1
              9 BINARY_MODULO
             10 STORE_FAST               0 (msg)

  5          13 LOAD_GLOBAL              1 (print)
             16 LOAD_FAST                0 (msg)
             19 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             22 POP_TOP
             23 LOAD_CONST               0 (None)
             26 RETURN_VALUE


In [5]:
# full implementation of the function
say_hello.__code__.co_code

b'd\x01\x00t\x00\x00f\x01\x00\x16}\x00\x00t\x01\x00|\x00\x00\x83\x01\x00\x01d\x00\x00S'

In [14]:
# func_closure
def hello_closure(what):
    msg = "Hello, %s" %(what,)
    def hello_closure_inner():
        return msg
    return hello_closure_inner

say_hello = hello_closure("World")

In [9]:
say_hello()

'Hello, World'

In [12]:
say_hello.__closure__

(<cell at 0x1061614c8: str object at 0x1061b20f0>,)

In [13]:
say_hello.__closure__[0].cell_contents

'Hello, World'

In [18]:
# back to this
# python overloads binary modulo operator to do string formatting
dis.disassemble(say_hello1.__code__)

  4           0 LOAD_CONST               1 ('Hello, %s')
              3 LOAD_GLOBAL              0 (what)
              6 BUILD_TUPLE              1
              9 BINARY_MODULO
             10 STORE_FAST               0 (msg)

  5          13 LOAD_GLOBAL              1 (print)
             16 LOAD_FAST                0 (msg)
             19 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             22 POP_TOP
             23 LOAD_CONST               0 (None)
             26 RETURN_VALUE


### An aside: Stack Machines
* Easy to implement
* Easy to reason about
* Easy to optimize

In [19]:
def in_():
    return "x" in ("x", )

In [22]:
def eq():
    return "x" == "x"

In [21]:
dis.disassemble(in_.__code__)

  2           0 LOAD_CONST               1 ('x')
              3 LOAD_CONST               2 (('x',))
              6 COMPARE_OP               6 (in)
              9 RETURN_VALUE


In [23]:
dis.disassemble(eq.__code__)

  2           0 LOAD_CONST               1 ('x')
              3 LOAD_CONST               1 ('x')
              6 COMPARE_OP               2 (==)
              9 RETURN_VALUE


Let's look at the source code:

[ceval.c](https://github.com/certik/python-2.7/blob/c360290c3c9e55fbd79d6ceacdfc7cd4f393c1eb/Python/ceval.c)
* This is where Python evaluation happens

Let's read through the switch.

> Why does the old @property operator not work with old style classes

[Blog post about string interning](http://guilload.com/python-string-interning/)

> Why can you not override `__len__`?
